In [1]:
import torch
import tiktoken
import matplotlib.pyplot as plt
from basic import create_dataloader_v1
from GPT_architecture import GPTModel
from GPT_architecture import Generate_text

In [2]:
GPT_CONFIG_124M = {
        "vocab_size": 50257,
        "context_length": 256, # Shortened from 1024 to make it easier to train on a laptop
        "emb_dim": 768,
        "n_heads": 12,
        "n_layers": 12,
        "drop_rate": 0.1,
        "qkv_bias": False
}

In [4]:
def text_to_token_ids(txt, tokenizer):
    encoded= tokenizer.encode(txt, allowed_special={'<|endoftext|>'})
    encoded_tensor= torch.tensor(encoded).unsqueeze(0) # add dimension at position 0
    return encoded_tensor

In [5]:
def token_ids_to_txt(token_ids, tokenizer):
    flat= token_ids.squeeze(0) # Remove batch dimension
    return tokenizer.decode(flat.tolist())

In [ ]:
def cross_entropy(data_loader, model, device, num_batches=None):
    ip_batch= ip_batch.to(device)
    target_batch= target_batch.to(device)
    logits= model(ip_batch)
    loss= torch.nn.functional.cross_entropy(
        logits.flatten(0,1), target_batch.flatten()
    )
    return loss

In [ ]:
def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss=0
    if len(data_loader)==0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches= min(num_batches, len(data_loader))
    
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i<num_batches:
            loss= cross_entropy(input_batch, target_batch, model, device)
            total_loss+=loss.item()
        else:
            break
    return total_loss/num_batches

In [ ]:
def evaulate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval() # to stop dropout
    # no grad to stop gradient calculation 
    with torch.no_grad(): 
        train_loss= cross_entropy(
            train_loader, model, device, num_batches=eval_iter
        )
        val_loss= cross_entropy(
            val_loader, model, device, num_batches=eval_iter
        )
    model.train()
    return train_loss, val_loss

In [ ]:
def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size= model.pos_emb.weight.shape[0]
    encoded= text_to_token_ids(start_context, tokenizer).to(device)
    with torch.no_grad():
        token_ids= Generate_text(model= model, idx= encoded, max_new_tokens=50, context_size=context_size)
        decoded_text= token_ids_to_txt(token_ids, tokenizer)
        print(decoded_text.replace("\n", " "))
    model.train()

In [ ]:
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs, eval_fr, eval_iter, start_context, tokenizer):
    train_losses, val_losses, track_token_seen= [], [], []
    tokens_seen, global_step= 0,-1
    for epoch in range(num_epochs):
        model.train()
        for input_batch, target_batch in train_loader:
         optimizer.zero_grad()
         loss = cross_entropy(input_batch, target_batch, model, device)
         loss.backward()
         optimizer.step()
         tokens_seen+=input_batch.numel()
         global_step+=1

         if global_step % eval_fr ==0:
            train_loss, val_loss = evaulate_model(model, train_loader, val_loader, device, eval_iter)
            train_losses.append(train_loss)
            val_losses.append(val_loss)
            print(f"Ep{epoch+1}(Step {global_step:06d}):"
                  f"Train loss{train_loss:.3f}"
                  f"Val loss{val_loss:.3f}")
        
        generate_and_print_sample(model, tokenizer, device, start_context)
    return train_losses, val_losses, track_token_seen

In [ ]:
def generate(model, idx, max_new_tokens, context_size, temperature=0.0, top_k=None, eos_id=None):
    for _ in range(max_new_tokens):
        idx_cond= idx[:, -context_size:]
        with torch.no_grad():
            logits= model(idx_cond)
        logits= logits[:,-1,:]
        if top_k is not None:
            top_logits, _=torch.topk(logits, top_k)
            min_val= top_logits[:,-1]
            logits= torch.where(
                logits<min_val,
                torch.tensor(float('-inf')).to(logits.device),
                logits
            )
        if temperature > 0.0:
            logits= logits/temperature
            probs= torch.softmax(logits, dim=-1)
            idx_next= torch.multinomial(probs, num_samples=1)
        else:
            idx_next= torch.argmax(logits, dim=-1, keepdim=True)
        if idx_next == eos_id:
            break
        idx= torch.cat((idx, idx_next), dim=1)
    return idx

In [ ]:
def plot_losses(epochs_seen, tokens_seen, train_losses, val_losses):
    fig, ax1 = plt.subplots(figsize=(5, 3))
    ax1.plot(epochs_seen, train_losses, label="Training loss")
    ax1.plot(epochs_seen, val_losses, linestyle="-.", label="Validation loss")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")
    ax1.legend(loc="upper right")
    ax2 = ax1.twiny()
    ax2.plot(tokens_seen, train_losses, alpha=0)
    ax2.set_xlabel("Tokens seen")
    fig.tight_layout()
    plt.show()

In [ ]:
def main():
    torch.manual_seed(123)
    model= GPTModel(GPT_CONFIG_124M)
    model.eval()

    start_context= "every effort moves you"
    tokenizer= tiktoken.get_encoding("gpt2")
    token_ids= Generate_text(
        model= model,
        idx= text_to_token_ids(start_context, tokenizer),
        max_new_tokens=10,
        context_size=GPT_CONFIG_124M['context_length']
    )
    print("Output text", token_ids_to_txt(token_ids, tokenizer))

    # Calculate loss
    inputs = torch.tensor([[16833, 3626, 6100], # ["every effort moves",
                           [40, 1107, 588]])    # "I really like"]
    
    targets = torch.tensor([[3626, 6100, 345],  # [" effort moves you",
                            [588, 428, 11311]]) # " really like chocholate"]
    with torch.no_grad():
        logits= model(inputs)
    probas= torch.softmax(logits, dim=-1)
    print('Probability received for next word:', probas.shape)

    token_ids= torch.argmax(probas, dim=-1, keepdim=True)
    print("Token IDs:", token_ids)

    print(f"Target batch 1: {token_ids_to_txt(targets[0], tokenizer)}")
    print(f"Output batch 1:" f"{token_ids_to_txt(token_ids[0].flatten(), tokenizer)}")

    text_idx=0
    target_probas_1= probas[text_idx, [0,1,2], targets[text_idx]]
    print("Text 1:", target_probas_1)

    text_idx = 1
    target_probas_2 = probas[text_idx, [0, 1, 2], targets[text_idx]]
    print("Text 2:", target_probas_2)

    log_probas= torch.log(torch.cat((target_probas_1, target_probas_2)))
    print("Log_probas:", log_probas)

    avg_log_probas= torch.mean(log_probas)
    print("Average log probas", avg_log_probas)

    neg_avg_log_probas= avg_log_probas*-1
    print("Negative acg log probas", neg_avg_log_probas)

    print("Logits shape:", logits.shape)
    print("target shape:", targets.shape)

    logits_flat= logits.flatten(0,1)
    targets_flat= targets.flatten()
    print("Flattened logits:", logits_flat.shape)
    print("Flattened targets", targets_flat.shape)

    loss= torch.nn.functional.binary_cross_entropy(logits.flat, targets_flat)
    print("loss", loss)

    perplexity= torch.exp(loss)
    print(f"Perplexity {perplexity}")

    filepath=
    with open (filepath, 'r', encoding='utf-8') as file:
        text_data= file.read()
    
    total_characters= len(text_data)
    total_tokens= len(tokenizer.encode(text_data))
    print("Characters:", total_characters)
    print("Tokens", total_tokens)

    train_ratio=0.90
    split_idx= int(train_ratio*len(text_data))
    train_data= text_data[:split_idx]
    val_data= text_data[split_idx]

    torch.manual_seed(123)

    train_loader= create_dataloader_v1(
        train_data,
        batch_size=2,
        max_len=GPT_CONFIG_124M['context_len'],
        stride= GPT_CONFIG_124M["context_len"],
        drop_last= True,
        shuffle= True,
        num_workers=0
    )
    val_loader = create_dataloader_v1(
            val_data,
            batch_size=2, # A more common batch size would be 1024, this is just for the demonstration purpose
            max_length=GPT_CONFIG_124M["context_length"],
            stride=GPT_CONFIG_124M["context_length"],
            drop_last=False,
            shuffle=False,
            num_workers=0
    )
    
    # Iterate through the data loaders to check that they were created correctly
    
    print("Train loader:")
    for x, y in train_loader:
        print(x.shape, y.shape)
    
    print("\nValidation loader:")
    for x, y in val_loader:
        print(x.shape, y.shape)
    
    device= torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    model.to(device)
    with torch.no_grad():
        train_loss= calc_loss_loader(train_loader, model, device)
        val_loss= calc_loss_loader(val_loader, model, device)
     
    print("Training loss:", train_loss)
    print("Val loss", val_loss)

    # Let's pretrain :)

    torch.manual_seed(123)
    model= GPTModel(GPT_CONFIG_124M)
    model.to(device)
    optimizer= torch.optim.AdamW(
        model.parameters(), lr=0.0004, weight_decay=0.1
    )    
    num_epochs= 10
    print("LET'S TRAIN")
    train_losses, val_losses, tokens_seen= train_model_simple(
        model, train_loader, val_loader, optimizer, device, num_epochs=num_epochs, eval_fr=5, eval_iter=1, start_context="Every effort moves you", tokenizer=tokenizer
    )
    epochs_tensor = torch.linspace(0, num_epochs, len(train_losses))
    plot_losses(epochs_tensor, tokens_seen, train_losses, val_losses)
    
    print("epochs tensor:", epochs_tensor)
    print("tokens seen:", tokens_seen)
    print("train losses:", train_losses)
    print("val losses:", val_losses)

    model.to("cpu")
    model.eval()

    tokenizer= tiktoken.get_encoding("gpt2")
    token_ids= Generate_text(model=model, idx= text_to_token_ids("Every effort moves you", tokenizer),
                            max_new_tokens=25,
                            context_size=GPT_CONFIG_124M["context_len"]
                             )
    
    torch.manual_seed(123)
    token_ids = generate(
            model=model,
            idx=text_to_token_ids("Every effort moves you", tokenizer),
            max_new_tokens=15,
            context_size=GPT_CONFIG_124M["context_length"],
            top_k=25,
            temperature=1.4
    )
    print("Output text:\n", token_ids_to_txt(token_ids, tokenizer))
    
    print("Saving the pretrained model")
    torch.save(model.state_dict(), "model.pth")
    
    print("Loading just saved model")
    
    model_loaded = GPTModel(GPT_CONFIG_124M)
    model_loaded.load_state_dict(torch.load("model.pth"))
    model_loaded.eval()
    
    token_ids = generate(
            model=model_loaded,
            idx=text_to_token_ids("It was another Thursday in the office", tokenizer),
            max_new_tokens=15,
            context_size=GPT_CONFIG_124M["context_length"],
            top_k=1,
            temperature=1.,
    )
    print("Output text:\n", token_ids_to_txt(token_ids, tokenizer))
    
    # Saving model together with optimizer to continue training from that step
    
    print("Saving model together with the optimizer now")
    torch.save({
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
        },
        "model_and_optimizer.pth"
    ) 

if __name__ == "__main__":
    main()